##### Copyright 2022 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Recommending movies: ranking

This tutorial is a slightly adapted version of the [basic ranking tutorial](https://www.tensorflow.org/recommenders/examples/basic_ranking) from TensorFlow Recommenders documentation.



## Imports


Let's first get our imports out of the way.

In [ ]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

## Preparing the dataset

We're continuing to use the MovieLens dataset. This time, we're also going to keep the ratings: these are the objectives we are trying to predict.

In [3]:
df = pd.read_csv("../data/users.csv", sep=",", encoding='utf-8')
df.id = df.id.astype(str)
ratings = tf.data.Dataset.from_tensor_slices(dict(df))

ratings = ratings.map(lambda x: {
    "store": x["store_name"],
    "user_id": x["id"],
    "user_rating": x["rating"]
})

We'll split the data by putting 80% of the ratings in the train set, and 20% in the test set.

In [4]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(10_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(8_000)
test = shuffled.skip(8_000).take(2_000)

Next we figure out unique user ids and movie titles present in the data so that we can create the embedding user and movie embedding tables.

In [5]:
movie_titles = ratings.batch(10_000).map(lambda x: x["store"])
user_ids = ratings.batch(10_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## Implementing a model

### Architecture

Ranking models do not face the same efficiency constraints as retrieval models do, and so we have a little bit more freedom in our choice of architectures. We can implement our ranking model as follows:

In [6]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])
    
  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

### Loss and metrics

We'll make use of the `Ranking` task object: a convenience wrapper that bundles together the loss function and metric computation. 

We'll use it together with the `MeanSquaredError` Keras loss in order to predict the ratings.

In [7]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

### The full model

We can now put it all together into a model.

In [8]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["store"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")
    
    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

## Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [9]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [10]:
cached_train = train.shuffle(10_000).batch(8192).cache()
cached_test = test.batch(1024).cache()

Then train the  model:

In [11]:
model.fit(cached_train, epochs=50)

Epoch 1/50
1/1 [==============================] - 1s 560ms/step - root_mean_squared_error: 3.9199 - loss: 15.3653 - regularization_loss: 0.0000e+00 - total_loss: 15.3653
Epoch 2/50
1/1 [==============================] - 0s 9ms/step - root_mean_squared_error: 2.9980 - loss: 8.9882 - regularization_loss: 0.0000e+00 - total_loss: 8.9882
Epoch 3/50
1/1 [==============================] - 0s 11ms/step - root_mean_squared_error: 1.5400 - loss: 2.3717 - regularization_loss: 0.0000e+00 - total_loss: 2.3717
Epoch 4/50
1/1 [==============================] - 0s 11ms/step - root_mean_squared_error: 2.6337 - loss: 6.9361 - regularization_loss: 0.0000e+00 - total_loss: 6.9361
Epoch 5/50
1/1 [==============================] - 0s 9ms/step - root_mean_squared_error: 1.1678 - loss: 1.3638 - regularization_loss: 0.0000e+00 - total_loss: 1.3638
Epoch 6/50
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 1.1676 - loss: 1.3632 - regularization_loss: 0.0000e+00 - total_loss: 1.36

As the model trains, the loss is falling and the RMSE metric is improving.

Finally, we can evaluate our model on the test set:

In [12]:
model.evaluate(cached_test, return_dict=True)

2/2 [==============================] - 0s 3ms/step - root_mean_squared_error: 1.1243 - loss: 1.2558 - regularization_loss: 0.0000e+00 - total_loss: 1.2558


{'root_mean_squared_error': 1.1243493556976318,
 'loss': 1.2400743961334229,
 'regularization_loss': 0,
 'total_loss': 1.2400743961334229}

The lower the RMSE metric, the more accurate our model is at predicting ratings.

## Exporting for serving

The model can be easily exported for serving:


In [13]:
tf.saved_model.save(model, "exported-ranking/123")

INFO:tensorflow:Assets written to: exported-ranking/123\assets


INFO:tensorflow:Assets written to: exported-ranking/123\assets



We will deploy the model with TensorFlow Serving soon.

In [ ]:
# Zip the SavedModel folder for easier download
!zip -r exported-ranking.zip exported-ranking/ 